# 1 导包和数据读取

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import seaborn as sns

# modelling
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score,cross_val_predict,KFold
from sklearn.metrics import make_scorer,mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures,MinMaxScaler,StandardScaler

In [20]:
#load_dataset
lanes_df = pd.read_csv('./data/Lane.csv')  
light_df = pd.read_csv('./data/Light_status.csv')
roads_df = pd.read_csv('./data/Entrance_road.csv') 
flow_df = pd.read_csv('./data/Flow.csv')

# 2 处理数据

## Lane.csv

In [21]:
lanes_df.head()

,LANE_ID,ARM_ID,LANE_NAME,DIR
0,1,E,左,L
1,1,N,左,L
2,1,S,左,L
3,1,W,左,L
4,2,E,直,S


In [22]:
lanes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   LANE_ID    16 non-null     int64 
 1   ARM_ID     16 non-null     object
 2   LANE_NAME  16 non-null     object
 3   DIR        16 non-null     object
dtypes: int64(1), object(3)
memory usage: 640.0+ bytes


## Light_status.csv

In [23]:
light_df = pd.read_csv('./data/Light_status.csv')
light_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105844 entries, 0 to 105843
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   CYCLE_START_TIME  105844 non-null  object
 1   STAGE_START_TIME  105844 non-null  object
 2   STAGE_END_TIME    105844 non-null  object
 3   STAGE_LENGTH      105844 non-null  int64 
 4   GREEN_TIME        105844 non-null  int64 
 5   GREEN_FLASH_TIME  105844 non-null  int64 
 6   YELLOW_TIME       105844 non-null  int64 
 7   ALL_RED_TIME      105844 non-null  int64 
 8   CHANNELS          105844 non-null  object
 9   LANES             105844 non-null  object
 10  PHASES            105844 non-null  object
 11  LANE_FUNCS        105844 non-null  object
dtypes: int64(5), object(7)
memory usage: 9.7+ MB


In [24]:
# 转换成日期属性
light_df['CYCLE_START_TIME'] = pd.to_datetime(light_df['CYCLE_START_TIME'])
light_df['STAGE_START_TIME'] = pd.to_datetime(light_df['STAGE_START_TIME'])
light_df['STAGE_END_TIME'] = pd.to_datetime(light_df['STAGE_END_TIME'])

In [25]:
# 删除GREEN_FLASH_TIME YELLOW_TIME列
light_df.drop('GREEN_FLASH_TIME',axis=1,inplace=True)
light_df.drop('YELLOW_TIME',axis=1,inplace=True)

In [26]:
# 查找STAGE_LENGTH或GREEN_TIME小于0的行，并删除
neg_stage_length =light_df[(light_df['STAGE_LENGTH']<0)|(light_df['GREEN_TIME']<0)]
light_df = light_df.drop(neg_stage_length.index,axis=0)
light_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105759 entries, 0 to 105843
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CYCLE_START_TIME  105759 non-null  datetime64[ns]
 1   STAGE_START_TIME  105759 non-null  datetime64[ns]
 2   STAGE_END_TIME    105759 non-null  datetime64[ns]
 3   STAGE_LENGTH      105759 non-null  int64         
 4   GREEN_TIME        105759 non-null  int64         
 5   ALL_RED_TIME      105759 non-null  int64         
 6   CHANNELS          105759 non-null  object        
 7   LANES             105759 non-null  object        
 8   PHASES            105759 non-null  object        
 9   LANE_FUNCS        105759 non-null  object        
dtypes: datetime64[ns](3), int64(3), object(4)
memory usage: 8.9+ MB


In [27]:
# 处理LANES列
light_df['released_lanes'] = light_df['LANES'].str.split(',') # 用，分割

In [28]:
# 将放行车道号“_”前代表的进口道，1，2，3，4替换为为W，N，E，S。

# 定义一个替换函数 replace_dir
def replace_lanes(lanes):
    replaced = []
    for i in lanes:
        i = i.replace('1_', 'W_')
        i = i.replace('2_', 'N_')
        i = i.replace('3_', 'E_')
        i = i.replace('4_', 'S_')
        
        replaced.append(i)
    
    return replaced

# apply替换
light_df['released_lanes'] = light_df['released_lanes'].apply(replace_lanes)

In [29]:
light_df['released_lanes']

0         [W_2, W_3, E_2, W_4, E_3, E_4]
1         [W_2, W_3, E_2, W_4, E_3, E_4]
2         [W_2, W_3, E_2, W_4, E_3, E_4]
3         [W_2, W_3, E_2, W_4, E_3, E_4]
4         [W_2, W_3, E_2, W_4, E_3, E_4]
                       ...              
105839                        [N_1, S_1]
105840                        [N_1, S_1]
105841                        [N_1, S_1]
105842                        [N_1, S_1]
105843                        [N_1, S_1]
Name: released_lanes, Length: 105759, dtype: object

In [30]:
# 调换顺序，改为前面为车道号，后面为进口道，并去掉下划线
light_df['released_lanes'] = light_df['released_lanes'].apply(lambda lanes: [lane.replace('_', '')[-1] + lane.replace('_', '')[:-1] for lane in lanes])

In [31]:
light_df.head(40000)

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS,released_lanes
0,2023-08-01 00:01:08,2023-08-01 00:01:08,2023-08-01 00:01:30,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
1,2023-08-01 00:02:18,2023-08-01 00:02:18,2023-08-01 00:02:40,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2,2023-08-01 00:04:38,2023-08-01 00:04:38,2023-08-01 00:05:00,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
3,2023-08-01 00:05:48,2023-08-01 00:05:48,2023-08-01 00:06:10,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
4,2023-08-01 00:06:58,2023-08-01 00:06:58,2023-08-01 00:07:20,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
...,...,...,...,...,...,...,...,...,...,...,...
40030,2023-08-11 07:07:18,2023-08-11 07:08:02,2023-08-11 07:08:26,24,21,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
40031,2023-08-11 07:08:43,2023-08-11 07:09:27,2023-08-11 07:09:51,24,21,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
40032,2023-08-11 07:10:08,2023-08-11 07:10:52,2023-08-11 07:11:16,24,21,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
40033,2023-08-11 07:11:33,2023-08-11 07:12:17,2023-08-11 07:12:58,41,38,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"


In [32]:
# 按LANES分组
lanes_unique = light_df['LANES'].unique()
lanes_unique

array(['1_2,1_3,3_2,1_4,3_3,3_4', '1_1,3_1', '2_2,2_3,2_4,4_2,4_3,4_4',
       '2_1,4_1'], dtype=object)

In [33]:
# 按LANES分组
df_ligth_2W3W2E4W3E4E=light_df[light_df['LANES']=='1_2,1_3,3_2,1_4,3_3,3_4']
df_ligth_1W1E=light_df[light_df['LANES']=='1_1,3_1']
df_ligth_2N3N4N2S3S4S=light_df[light_df['LANES']=='2_2,2_3,2_4,4_2,4_3,4_4']
df_ligth_1N1S=light_df[light_df['LANES']=='2_1,4_1']

In [34]:
df_ligth_2W3W2E4W3E4E

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS,released_lanes
0,2023-08-01 00:01:08,2023-08-01 00:01:08,2023-08-01 00:01:30,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
1,2023-08-01 00:02:18,2023-08-01 00:02:18,2023-08-01 00:02:40,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2,2023-08-01 00:04:38,2023-08-01 00:04:38,2023-08-01 00:05:00,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
3,2023-08-01 00:05:48,2023-08-01 00:05:48,2023-08-01 00:06:10,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
4,2023-08-01 00:06:58,2023-08-01 00:06:58,2023-08-01 00:07:20,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
...,...,...,...,...,...,...,...,...,...,...,...
71245,2023-08-27 23:55:01,2023-08-27 23:55:01,2023-08-27 23:55:23,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
71246,2023-08-27 23:56:11,2023-08-27 23:56:11,2023-08-27 23:56:33,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
71247,2023-08-27 23:57:21,2023-08-27 23:57:21,2023-08-27 23:57:43,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
71248,2023-08-27 23:58:31,2023-08-27 23:58:31,2023-08-27 23:58:53,22,19,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"


In [35]:
# 删除STAGE_START_TIME，STAGE_END_TIME，LANES
df_ligth_2W3W2E4W3E4E.drop(['STAGE_START_TIME','STAGE_END_TIME','LANES'],axis=1,inplace=True)

In [36]:
#处理时间索引，按CYCLE_START_TIME为时间索引，从00:05:00开始，每五分钟为一个时间段

In [37]:
# 删除2023-08-01 00:05:00前的数据
df_ligth_2W3W2E4W3E4E=df_ligth_2W3W2E4W3E4E.drop(index=df_ligth_2W3W2E4W3E4E.index[:3])
df_ligth_2W3W2E4W3E4E=df_ligth_2W3W2E4W3E4E.reset_index(drop=True)
df_ligth_2W3W2E4W3E4E

,CYCLE_START_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
0,2023-08-01 00:05:48,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
1,2023-08-01 00:06:58,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2,2023-08-01 00:08:08,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
3,2023-08-01 00:09:18,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
4,2023-08-01 00:10:28,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
...,...,...,...,...,...,...,...,...
26447,2023-08-27 23:55:01,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
26448,2023-08-27 23:56:11,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
26449,2023-08-27 23:57:21,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
26450,2023-08-27 23:58:31,22,19,0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"


In [38]:
# 将第一个数据改成2023-08-01 00:05:00
#df_ligth_2W3W2E4W3E4E['CYCLE_START_TIME'][0]=pd.to_datetime("2023-08-01 00:05:00")
#df_ligth_2W3W2E4W3E4E.head()

In [39]:
# 将CYCLE_START_TIME设置为时间索引
df_ligth_2W3W2E4W3E4E=df_ligth_2W3W2E4W3E4E.set_index('CYCLE_START_TIME')
#df_ligth_2W3W2E4W3E4E.head(20)

In [40]:
# 对数据进行重新采样，以五分钟为一个时间段，并选择从00:05:00开始的数据：
start_time = pd.to_datetime("2023-08-01 00:05:00")
df_ligth_2W3W2E4W3E4E = df_ligth_2W3W2E4W3E4E.resample('5Min',label='left', closed='right').first().loc[start_time:] 

In [41]:
df_ligth_2W3W2E4W3E4E

,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
CYCLE_START_TIME,,,,,,,
2023-08-01 00:05:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:10:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:15:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:20:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:25:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
...,...,...,...,...,...,...,...
2023-08-27 23:35:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-27 23:40:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-27 23:45:00,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"


In [42]:
df_ligth_1W1E.head(10)

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS,released_lanes
14944,2023-07-31 23:59:58,2023-08-01 00:00:20,2023-08-01 00:00:33,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14945,2023-08-01 00:01:08,2023-08-01 00:01:30,2023-08-01 00:01:43,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14946,2023-08-01 00:02:18,2023-08-01 00:02:40,2023-08-01 00:02:53,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14947,2023-08-01 00:02:18,2023-08-01 00:03:50,2023-08-01 00:04:03,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14948,2023-08-01 00:04:38,2023-08-01 00:05:00,2023-08-01 00:05:13,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14949,2023-08-01 00:05:48,2023-08-01 00:06:10,2023-08-01 00:06:23,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14950,2023-08-01 00:06:58,2023-08-01 00:07:20,2023-08-01 00:07:33,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14951,2023-08-01 00:08:08,2023-08-01 00:08:30,2023-08-01 00:08:43,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14952,2023-08-01 00:09:18,2023-08-01 00:09:40,2023-08-01 00:09:53,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"
14953,2023-08-01 00:10:28,2023-08-01 00:10:50,2023-08-01 00:11:03,13,9,1,"1,9","1_1,3_1","4,9",12,"[1W, 1E]"


In [43]:
# 删除STAGE_START_TIME，STAGE_END_TIME，LANES
df_ligth_1W1E.drop(['STAGE_START_TIME','STAGE_END_TIME','LANES'],axis=1,inplace=True)

In [44]:
# 删除2023-08-01 00:05:00前的数据
df_ligth_1W1E = df_ligth_1W1E.drop(index=df_ligth_1W1E.index[:5])
df_ligth_1W1E=df_ligth_1W1E.reset_index(drop=True)
df_ligth_1W1E.head()

,CYCLE_START_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
0,2023-08-01 00:05:48,13,9,1,"1,9","4,9",12,"[1W, 1E]"
1,2023-08-01 00:06:58,13,9,1,"1,9","4,9",12,"[1W, 1E]"
2,2023-08-01 00:08:08,13,9,1,"1,9","4,9",12,"[1W, 1E]"
3,2023-08-01 00:09:18,13,9,1,"1,9","4,9",12,"[1W, 1E]"
4,2023-08-01 00:10:28,13,9,1,"1,9","4,9",12,"[1W, 1E]"


In [45]:
# 将第一条数据改成2023-08-01 00:05:00
#df_ligth_1W1E['CYCLE_START_TIME'][0]=pd.to_datetime('2023-08-01 00:05:00')

In [46]:
# 设置时间索引
df_ligth_1W1E=df_ligth_1W1E.set_index("CYCLE_START_TIME")
df_ligth_1W1E.head()

,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
CYCLE_START_TIME,,,,,,,
2023-08-01 00:05:48,13,9,1,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:06:58,13,9,1,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:08:08,13,9,1,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:09:18,13,9,1,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:10:28,13,9,1,"1,9","4,9",12,"[1W, 1E]"


In [47]:
# 对数据进行重新采样，以五分钟为一个时间段，并选择从00:05:00开始的数据：
df_ligth_1W1E = df_ligth_1W1E.resample('5Min',label='left',closed = 'right').first().loc[start_time:]

In [48]:
df_ligth_1W1E

,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
CYCLE_START_TIME,,,,,,,
2023-08-01 00:05:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:10:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:15:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:20:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
2023-08-01 00:25:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
...,...,...,...,...,...,...,...
2023-08-27 23:35:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
2023-08-27 23:40:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"
2023-08-27 23:45:00,13.0,9.0,1.0,"1,9","4,9",12,"[1W, 1E]"


In [49]:
df_ligth_2N3N4N2S3S4S.head(10)

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS,released_lanes
29861,2023-07-31 23:59:58,2023-08-01 00:00:33,2023-08-01 00:00:55,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29862,2023-08-01 00:01:08,2023-08-01 00:01:43,2023-08-01 00:02:05,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29863,2023-08-01 00:02:18,2023-08-01 00:02:53,2023-08-01 00:03:15,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29864,2023-08-01 00:02:18,2023-08-01 00:04:03,2023-08-01 00:04:25,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29865,2023-08-01 00:04:38,2023-08-01 00:05:13,2023-08-01 00:05:35,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29866,2023-08-01 00:05:48,2023-08-01 00:06:23,2023-08-01 00:06:58,35,32,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29867,2023-08-01 00:06:58,2023-08-01 00:07:33,2023-08-01 00:07:55,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29868,2023-08-01 00:08:08,2023-08-01 00:08:43,2023-08-01 00:09:05,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29869,2023-08-01 00:09:18,2023-08-01 00:09:53,2023-08-01 00:10:15,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
29870,2023-08-01 00:10:28,2023-08-01 00:11:03,2023-08-01 00:11:25,22,19,0,"14,15,6,7","2_2,2_3,2_4,4_2,4_3,4_4","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"


In [50]:
# 删除STAGE_START_TIME，STAGE_END_TIME，LANES
df_ligth_2N3N4N2S3S4S.drop(['STAGE_START_TIME','STAGE_END_TIME','LANES'],axis=1,inplace=True)

In [51]:
# 删除数据 处理索引
df_ligth_2N3N4N2S3S4S = df_ligth_2N3N4N2S3S4S.drop(index=df_ligth_2N3N4N2S3S4S.index[:5])
df_ligth_2N3N4N2S3S4S = df_ligth_2N3N4N2S3S4S.reset_index(drop=True)

In [52]:
# 修改数据
#df_ligth_2N3N4N2S3S4S['CYCLE_START_TIME'][0]=pd.to_datetime('2023-08-01 00:05:00')

In [53]:
# 设置时间索引
df_ligth_2N3N4N2S3S4S=df_ligth_2N3N4N2S3S4S.set_index("CYCLE_START_TIME")

In [54]:
# 重新采样
df_ligth_2N3N4N2S3S4S=df_ligth_2N3N4N2S3S4S.resample('5Min',label='left',closed='right').first().loc[start_time:]

In [55]:
df_ligth_2N3N4N2S3S4S

,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
CYCLE_START_TIME,,,,,,,
2023-08-01 00:05:00,35.0,32.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
2023-08-01 00:10:00,22.0,19.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
2023-08-01 00:15:00,22.0,19.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
2023-08-01 00:20:00,23.0,20.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
2023-08-01 00:25:00,22.0,19.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
...,...,...,...,...,...,...,...
2023-08-27 23:35:00,22.0,19.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
2023-08-27 23:40:00,22.0,19.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"
2023-08-27 23:45:00,22.0,19.0,0.0,"14,15,6,7","2,6,8","11,13","[2N, 3N, 4N, 2S, 3S, 4S]"


In [56]:
df_ligth_1N1S.head(10)

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS,released_lanes
44812,2023-07-31 23:59:58,2023-08-01 00:00:55,2023-08-01 00:01:08,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44813,2023-08-01 00:01:08,2023-08-01 00:02:05,2023-08-01 00:02:18,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44814,2023-08-01 00:02:18,2023-08-01 00:03:15,2023-08-01 00:03:50,35,31,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44815,2023-08-01 00:02:18,2023-08-01 00:04:25,2023-08-01 00:04:38,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44816,2023-08-01 00:04:38,2023-08-01 00:05:35,2023-08-01 00:05:48,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44817,2023-08-01 00:06:58,2023-08-01 00:07:55,2023-08-01 00:08:08,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44818,2023-08-01 00:08:08,2023-08-01 00:09:05,2023-08-01 00:09:18,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44819,2023-08-01 00:09:18,2023-08-01 00:10:15,2023-08-01 00:10:28,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44820,2023-08-01 00:10:28,2023-08-01 00:11:25,2023-08-01 00:11:38,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"
44821,2023-08-01 00:11:38,2023-08-01 00:12:35,2023-08-01 00:12:48,13,9,1,"13,5","2_1,4_1","1,7",12,"[1N, 1S]"


In [57]:
# 删除STAGE_START_TIME，STAGE_END_TIME，LANES
df_ligth_1N1S.drop(['STAGE_START_TIME','STAGE_END_TIME','LANES'],axis=1,inplace=True)

In [58]:
# 删除数据 整理索引
df_ligth_1N1S = df_ligth_1N1S.drop(index=df_ligth_1N1S.index[:4])
df_ligth_1N1S=df_ligth_1N1S.reset_index(drop=True)

In [59]:
# 修改数据
#df_ligth_1N1S['CYCLE_START_TIME'][0]=pd.to_datetime("2023-08-01 00:05:00")

In [60]:
# 设置时间索引
df_ligth_1N1S = df_ligth_1N1S.set_index("CYCLE_START_TIME")

In [61]:
# 重新采样
df_ligth_1N1S = df_ligth_1N1S.resample("5Min",label="left",closed="right").first().loc[start_time:]

In [62]:
df_ligth_1N1S

,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
CYCLE_START_TIME,,,,,,,
2023-08-01 00:05:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"
2023-08-01 00:10:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"
2023-08-01 00:15:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"
2023-08-01 00:20:00,12.0,8.0,1.0,"13,5","1,7",12,"[1N, 1S]"
2023-08-01 00:25:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"
...,...,...,...,...,...,...,...
2023-08-27 23:35:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"
2023-08-27 23:40:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"
2023-08-27 23:45:00,13.0,9.0,1.0,"13,5","1,7",12,"[1N, 1S]"


## Entrance_road.csv

In [63]:
roads_df.head()

,ARM_ID,ARM_NAME,DIR
0,E,东进口道,E
1,N,北进口道,N
2,S,南进口道,S
3,W,西进口道,W


## Flow.csv

In [64]:
flow_df

,LANE_ID,ARM_ID,START_TIME,END_TIME,VOLUMN_5MIN
0,1,E,2023-08-01 00:05:00,2023-08-01 00:10:00,2
1,1,E,2023-08-01 00:10:00,2023-08-01 00:15:00,1
2,1,E,2023-08-01 00:15:00,2023-08-01 00:20:00,0
3,1,E,2023-08-01 00:20:00,2023-08-01 00:25:00,1
4,1,E,2023-08-01 00:25:00,2023-08-01 00:30:00,0
...,...,...,...,...,...
120775,4,W,2023-08-27 23:35:00,2023-08-27 23:40:00,0
120776,4,W,2023-08-27 23:40:00,2023-08-27 23:45:00,0
120777,4,W,2023-08-27 23:45:00,2023-08-27 23:50:00,0
120778,4,W,2023-08-27 23:50:00,2023-08-27 23:55:00,0


In [65]:
flow_df['START_TIME'] = pd.to_datetime(flow_df['START_TIME'])
flow_df['END_TIME'] = pd.to_datetime(flow_df['END_TIME'])

In [66]:
flow_df['LANE_ARM']=flow_df['LANE_ID'].astype(str)+flow_df['ARM_ID']

In [67]:
lane_arm = flow_df.pop("LANE_ARM")
flow_df.insert(2,'LANE_ARM',lane_arm)

In [68]:
flow_df

,LANE_ID,ARM_ID,LANE_ARM,START_TIME,END_TIME,VOLUMN_5MIN
0,1,E,1E,2023-08-01 00:05:00,2023-08-01 00:10:00,2
1,1,E,1E,2023-08-01 00:10:00,2023-08-01 00:15:00,1
2,1,E,1E,2023-08-01 00:15:00,2023-08-01 00:20:00,0
3,1,E,1E,2023-08-01 00:20:00,2023-08-01 00:25:00,1
4,1,E,1E,2023-08-01 00:25:00,2023-08-01 00:30:00,0
...,...,...,...,...,...,...
120775,4,W,4W,2023-08-27 23:35:00,2023-08-27 23:40:00,0
120776,4,W,4W,2023-08-27 23:40:00,2023-08-27 23:45:00,0
120777,4,W,4W,2023-08-27 23:45:00,2023-08-27 23:50:00,0
120778,4,W,4W,2023-08-27 23:50:00,2023-08-27 23:55:00,0


In [69]:
LANE_ARM_uniqe =flow_df['LANE_ARM'].unique()
LANE_ARM_uniqe

array(['1E', '1N', '1S', '1W', '2E', '2N', '2S', '2W', '3E', '3N', '3S',
       '3W', '4E', '4N', '4S', '4W'], dtype=object)

In [70]:
# 按LANE_ARM分组
df_flow_1E = flow_df[flow_df['LANE_ARM']=='1E']
df_flow_1N = flow_df[flow_df['LANE_ARM']=='1N']
df_flow_1S = flow_df[flow_df['LANE_ARM']=='1S']
df_flow_1W = flow_df[flow_df['LANE_ARM']=='1W']
df_flow_2E = flow_df[flow_df['LANE_ARM']=='2E']
df_flow_2N = flow_df[flow_df['LANE_ARM']=='2N']
df_flow_2S = flow_df[flow_df['LANE_ARM']=='2S']
df_flow_2W = flow_df[flow_df['LANE_ARM']=='2W']
df_flow_3E = flow_df[flow_df['LANE_ARM']=='3E']
df_flow_3N = flow_df[flow_df['LANE_ARM']=='3N']
df_flow_3S = flow_df[flow_df['LANE_ARM']=='3S']
df_flow_3W = flow_df[flow_df['LANE_ARM']=='3W']
df_flow_4E = flow_df[flow_df['LANE_ARM']=='4E']
df_flow_4N = flow_df[flow_df['LANE_ARM']=='4N']
df_flow_4S = flow_df[flow_df['LANE_ARM']=='4S']
df_flow_4W = flow_df[flow_df['LANE_ARM']=='4W']

In [71]:
flow_LANE_ARM = [df_flow_1E
,df_flow_1N
,df_flow_1S
,df_flow_1W
,df_flow_2E
,df_flow_2N
,df_flow_2S
,df_flow_2W
,df_flow_3E
,df_flow_3N
,df_flow_3S
,df_flow_3W
,df_flow_4E
,df_flow_4N
,df_flow_4S
,df_flow_4W]

In [72]:
for i in flow_LANE_ARM:
    print(i.shape)
    #i = i.reset_index(drop=True)


(7549, 6)
(7549, 6)
(7549, 6)
(7548, 6)
(7549, 6)
(7549, 6)
(7549, 6)
(7548, 6)
(7549, 6)
(7549, 6)
(7549, 6)
(7548, 6)
(7549, 6)
(7549, 6)
(7549, 6)
(7548, 6)


In [73]:
df_flow_1E = df_flow_1E.set_index('START_TIME')
df_flow_1N = df_flow_1N.set_index('START_TIME')
df_flow_1S = df_flow_1S.set_index('START_TIME')
df_flow_1W = df_flow_1W.set_index('START_TIME')
df_flow_2E = df_flow_2E.set_index('START_TIME')
df_flow_2N = df_flow_2N.set_index('START_TIME')
df_flow_2S = df_flow_2S.set_index('START_TIME')
df_flow_2W = df_flow_2W.set_index('START_TIME')
df_flow_3E = df_flow_3E.set_index('START_TIME')
df_flow_3N = df_flow_3N.set_index('START_TIME')
df_flow_3S = df_flow_3S.set_index('START_TIME')
df_flow_3W = df_flow_3W.set_index('START_TIME')
df_flow_4E = df_flow_4E.set_index('START_TIME')
df_flow_4N = df_flow_4N.set_index('START_TIME')
df_flow_4S = df_flow_4S.set_index('START_TIME')
df_flow_4W = df_flow_4W.set_index('START_TIME')

In [74]:
df_flow_1E.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_1N.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_1S.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_1W.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_2E.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_2N.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_2S.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_2W.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_3E.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_3N.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_3S.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_3W.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_4E.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_4N.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_4S.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)
df_flow_4W.drop(['LANE_ID','ARM_ID'],axis=1,inplace=True)

In [75]:
df_flow_4E

,LANE_ARM,END_TIME,VOLUMN_5MIN
START_TIME,,,
2023-08-01 00:05:00,4E,2023-08-01 00:10:00,0
2023-08-01 00:10:00,4E,2023-08-01 00:15:00,0
2023-08-01 00:15:00,4E,2023-08-01 00:20:00,0
2023-08-01 00:20:00,4E,2023-08-01 00:25:00,2
2023-08-01 00:25:00,4E,2023-08-01 00:30:00,0
...,...,...,...
2023-08-27 23:35:00,4E,2023-08-27 23:40:00,0
2023-08-27 23:40:00,4E,2023-08-27 23:45:00,0
2023-08-27 23:45:00,4E,2023-08-27 23:50:00,0


### 合并数据

In [79]:
#合并lanes_df和flow_df
#merged_df = pd.merge(lanes_df, flow_df, on=['LANE_ID', 'ARM_ID'])

In [80]:
# 合并
#merged_df_flow_4E = pd.merge(df_flow_4E,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='outer')
#merged_df_flow_4E

In [81]:
#df_ligth_1N1S
#df_ligth_2N3N4N2S3S4S
#df_ligth_1W1E
#df_ligth_2W3W2E4W3E4E

In [84]:
merged_df_flow_1E = pd.merge(df_flow_1E,df_ligth_1W1E, left_index=True, right_index=True, how='left')
merged_df_flow_1N = pd.merge(df_flow_1N,df_ligth_1N1S, left_index=True, right_index=True, how='left')
merged_df_flow_1S = pd.merge(df_flow_1S,df_ligth_1N1S, left_index=True, right_index=True, how='left')
merged_df_flow_1W = pd.merge(df_flow_1W,df_ligth_1W1E, left_index=True, right_index=True, how='left')
merged_df_flow_2E = pd.merge(df_flow_2E,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='left')
merged_df_flow_2N = pd.merge(df_flow_2N,df_ligth_2N3N4N2S3S4S, left_index=True, right_index=True, how='left')
merged_df_flow_2S = pd.merge(df_flow_2S,df_ligth_2N3N4N2S3S4S, left_index=True, right_index=True, how='left')
merged_df_flow_2W = pd.merge(df_flow_2W,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='left')
merged_df_flow_3E = pd.merge(df_flow_3E,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='left')
merged_df_flow_3N = pd.merge(df_flow_3N,df_ligth_2N3N4N2S3S4S, left_index=True, right_index=True, how='left')
merged_df_flow_3S = pd.merge(df_flow_3S,df_ligth_2N3N4N2S3S4S, left_index=True, right_index=True, how='left')
merged_df_flow_3W = pd.merge(df_flow_3W,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='left')
merged_df_flow_4E = pd.merge(df_flow_4E,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='left')
merged_df_flow_4N = pd.merge(df_flow_4N,df_ligth_2N3N4N2S3S4S, left_index=True, right_index=True, how='left')
merged_df_flow_4S = pd.merge(df_flow_4S,df_ligth_2N3N4N2S3S4S, left_index=True, right_index=True, how='left')
merged_df_flow_4W = pd.merge(df_flow_4W,df_ligth_2W3W2E4W3E4E, left_index=True, right_index=True, how='left')

In [85]:
merged_df_flow_2E

,LANE_ARM,END_TIME,VOLUMN_5MIN,STAGE_LENGTH,GREEN_TIME,ALL_RED_TIME,CHANNELS,PHASES,LANE_FUNCS,released_lanes
START_TIME,,,,,,,,,,
2023-08-01 00:05:00,2E,2023-08-01 00:10:00,0,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:10:00,2E,2023-08-01 00:15:00,1,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:15:00,2E,2023-08-01 00:20:00,0,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:20:00,2E,2023-08-01 00:25:00,1,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-01 00:25:00,2E,2023-08-01 00:30:00,0,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
...,...,...,...,...,...,...,...,...,...,...
2023-08-27 23:35:00,2E,2023-08-27 23:40:00,0,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-27 23:40:00,2E,2023-08-27 23:45:00,0,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"
2023-08-27 23:45:00,2E,2023-08-27 23:50:00,0,22.0,19.0,0.0,"11,2,3,10","3,5,10","11,13","[2W, 3W, 2E, 4W, 3E, 4E]"


# 3 初步训练模型

In [ ]:
# Data preprocessing

# Load and process 4 CSVs (code from previous examples) 

# Train/val/test split
VAL_PCT = 0.2 # validation percent
TEST_PCT = 0.2 # test percent

val_size = int(len(df) * VAL_PCT)
test_size = int(len(df) * TEST_PCT)

df_train = df[:-(val_size+test_size)] 
df_val = df[-(val_size+test_size):-test_size]
df_test = df[-test_size:]

# Scale target column
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df_train['traffic_volume'].values.reshape(-1,1))

df_train['traffic_volume'] = scaler.transform(df_train['traffic_volume'].values.reshape(-1,1))
df_val['traffic_volume'] = scaler.transform(df_val['traffic_volume'].values.reshape(-1,1)) 
df_test['traffic_volume'] = scaler.transform(df_test['traffic_volume'].values.reshape(-1,1))

# Create sequences
SEQ_LEN = 12 # sequence length

X_train, y_train = [], []
for i in range(SEQ_LEN, len(df_train)):
    X_train.append(df_train.iloc[i-SEQ_LEN:i])
    y_train.append(df_train.iloc[i]['traffic_volume'])
    
X_val, y_val = [], [] 
# similarly for df_val

X_test, y_test = [], []
# similarly for df_test

# Build RNN model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(X_train[0].shape[0], X_train[0].shape[1])))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

# Train model 
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val)) 

# Evaluate on test set
y_pred = model.predict(X_test)
# Inverse transform predictions
y_pred = scaler.inverse_transform(y_pred) 

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('Test MSE:', mse)